# Event Score Validation

**Objective:** Validate that pressure_score distinguishes information vs liquidity pressure economically.

**Week 2 Deliverable**

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
%matplotlib inline

## Load Event Features

In [ ]:
ev = pd.read_parquet('../data/processed/event_features.parquet')

print(f"Events: {len(ev):,}")
print(f"Date range: {ev['event_date'].min()} to {ev['event_date'].max()}")
print(f"\nColumns: {list(ev.columns)}")

---

## 1. Score Distribution

Check if pressure_score spans the full range and has reasonable variance.

In [ ]:
# Summary statistics
print("Pressure Score Distribution:")
print(ev['pressure_score'].describe())

print("\nInfo Score Distribution:")
print(ev['info_score'].describe())

print("\nLiquidity Score Distribution:")
print(ev['liq_score'].describe())

In [ ]:
# Histograms
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(ev['pressure_score'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Pressure Score')
axes[0].set_ylabel('Count')
axes[0].set_title('Pressure Score Distribution')
axes[0].axvline(0, color='red', linestyle='--', alpha=0.5, label='Neutral')
axes[0].legend()

axes[1].hist(ev['info_score'], bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1].set_xlabel('Info Score')
axes[1].set_ylabel('Count')
axes[1].set_title('Information Score')

axes[2].hist(ev['liq_score'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[2].set_xlabel('Liquidity Score')
axes[2].set_ylabel('Count')
axes[2].set_title('Liquidity Score')

plt.tight_layout()
plt.show()

print("✓ Scores span reasonable range")
print(f"  Pressure: [{ev['pressure_score'].min():.2f}, {ev['pressure_score'].max():.2f}]")
print(f"  Info: [{ev['info_score'].min():.2f}, {ev['info_score'].max():.2f}]")
print(f"  Liquidity: [{ev['liq_score'].min():.2f}, {ev['liq_score'].max():.2f}]")

**Expected:**
- Pressure score centered near 0 (or slight info bias for earnings)
- Good spread (not all clustered)
- Range uses most of [-1, +1]

---

## 2. Extreme Events Inspection

**Critical Test:** Do top/bottom 5% events show expected economic patterns?

In [ ]:
# Top 5% (High info pressure)
top_5pct = ev.nlargest(int(len(ev)*0.05), 'pressure_score')

# Bottom 5% (High liquidity pressure)
bot_5pct = ev.nsmallest(int(len(ev)*0.05), 'pressure_score')

print(f"Top 5%: {len(top_5pct)} events")
print(f"Bottom 5%: {len(bot_5pct)} events")

In [ ]:
# Display columns of interest
display_cols = [
    'event_id', 'permno', 'event_date', 
    'pressure_score', 'info_score', 'liq_score',
    'ofi_mean', 'ofi_autocorr', 'spread_stability_mean',
    'ofi_abs_mean', 'volume_burst_fraction', 'spread_p95_max', 'spread_std_mean'
]

print("\n" + "="*80)
print("TOP 5% - INFORMATION PRESSURE (Should show persistent flow, stable spreads)")
print("="*80)
display(top_5pct[display_cols].head(10))

print("\n" + "="*80)
print("BOTTOM 5% - LIQUIDITY PRESSURE (Should show volume spikes, spread widening)")
print("="*80)
display(bot_5pct[display_cols].head(10))

In [ ]:
# Compare average characteristics
comparison_cols = [
    'ofi_autocorr', 'spread_stability_mean',
    'ofi_abs_mean', 'volume_burst_fraction', 
    'spread_p95_max', 'spread_std_mean'
]

comparison = pd.DataFrame({
    'Top 5% (Info)': top_5pct[comparison_cols].mean(),
    'Bottom 5% (Liq)': bot_5pct[comparison_cols].mean(),
    'All Events': ev[comparison_cols].mean()
})

comparison['Info > Liq'] = comparison['Top 5% (Info)'] > comparison['Bottom 5% (Liq)']

print("\nCharacteristics Comparison:")
print(comparison)

print("\n" + "="*80)
print("VALIDATION CHECKS:")
print("="*80)

# Expected patterns
checks = {
    'ofi_autocorr higher in info': comparison.loc['ofi_autocorr', 'Info > Liq'],
    'spread_stability higher in info': comparison.loc['spread_stability_mean', 'Info > Liq'],
    'volume_burst_fraction lower in info': not comparison.loc['volume_burst_fraction', 'Info > Liq'],
    'spread_p95 lower in info': not comparison.loc['spread_p95_max', 'Info > Liq']
}

for check, passed in checks.items():
    status = "✓ PASS" if passed else "✗ FAIL"
    print(f"{status}: {check}")

total_passed = sum(checks.values())
print(f"\nOverall: {total_passed}/{len(checks)} checks passed")

if total_passed >= 3:
    print("\n✅ CLASSIFIER SHOWS EXPECTED ECONOMIC PATTERNS")
else:
    print("\n⚠️  WARNING: Classifier may not be separating correctly")

**Expected Patterns:**

**Top 5% (Info pressure):**
- High `ofi_autocorr` (persistent directional flow)
- High `spread_stability` (stable spreads)
- Low `volume_burst_fraction` (no abnormal volume)

**Bottom 5% (Liquidity pressure):**
- High `volume_burst_fraction` (volume spikes)
- High `spread_p95` (spread widening)
- High `ofi_abs` (aggressive trading)

If patterns don't match → classifier design issue.

---

## 3. Component Contribution

**Risk:** Is pressure_score dominated by a single component?

In [ ]:
# Check raw component correlations with pressure_score
component_cols = [
    'ofi_mean', 'ofi_autocorr', 'spread_stability_mean',  # Info
    'ofi_abs_mean', 'volume_burst_fraction', 'spread_std_mean', 'spread_p95_max'  # Liq
]

corrs = ev[component_cols + ['pressure_score', 'info_score', 'liq_score']].corr()

print("Component Correlations with Pressure Score:")
print(corrs['pressure_score'].sort_values(ascending=False))

print("\nComponent Correlations with Info Score:")
print(corrs['info_score'].sort_values(ascending=False))

print("\nComponent Correlations with Liq Score:")
print(corrs['liq_score'].sort_values(ascending=False))

In [ ]:
# Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corrs, annot=True, fmt='.2f', cmap='RdBu_r', center=0, 
            vmin=-1, vmax=1, square=True, cbar_kws={'label': 'Correlation'})
plt.title('Component Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Check for dominance
pressure_corrs = corrs['pressure_score'][component_cols].abs()

print("\n" + "="*80)
print("DOMINANCE CHECK")
print("="*80)

max_corr = pressure_corrs.max()
dominant_component = pressure_corrs.idxmax()

print(f"Strongest correlation: {dominant_component} ({max_corr:.3f})")

if max_corr > 0.95:
    print("\n⚠️  WARNING: Single component dominates (corr > 0.95)")
    print("   → Consider removing or downweighting this component")
elif max_corr > 0.85:
    print("\n⚠️  CAUTION: One component has high influence (corr > 0.85)")
    print("   → Acceptable but watch for redundancy")
else:
    print("\n✅ GOOD: No single component dominates")
    print(f"   All correlations < 0.85")

print("\nAll component correlations:")
for comp in component_cols:
    corr_val = corrs.loc[comp, 'pressure_score']
    print(f"  {comp:30s}: {corr_val:6.3f}")

**Interpretation:**
- **corr > 0.95:** Red flag - one component dominates
- **0.3 < corr < 0.85:** Good - balanced contribution
- **corr < 0.3:** Component may not be contributing much

**Goal:** All components contribute meaningfully, none dominate.

---

## Summary & Conclusions

In [ ]:
print("="*80)
print("VALIDATION SUMMARY")
print("="*80)

print(f"\n1. Score Distribution:")
print(f"   Events: {len(ev):,}")
print(f"   Pressure range: [{ev['pressure_score'].min():.2f}, {ev['pressure_score'].max():.2f}]")
print(f"   Std: {ev['pressure_score'].std():.3f}")

print(f"\n2. Extreme Events:")
print(f"   Economic patterns: {total_passed}/{len(checks)} checks passed")

print(f"\n3. Component Contribution:")
print(f"   Max component correlation: {max_corr:.3f}")
print(f"   Dominant component: {dominant_component}")

print("\n" + "="*80)
print("WEEK 2 STATUS: ✅ EVENT FEATURES VALIDATED")
print("="*80)
print("\nReady for Week 3: Event Study + Returns Analysis")

---

## Next Steps (Week 3)

1. Load forward returns [0, +5], [0, +20]
2. Quintile sorts by pressure_score
3. Test hypothesis:
   - High info pressure → drift
   - High liquidity pressure → reversion
4. Fama-MacBeth regressions with controls